In [1]:
from jitcdde import jitcdde, y, t
from parameters_expanded import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# initialize state variables, naming convention: [coponent_(material->material)[loop][index]][_(material)[index]]
# e.g. fuel->helium heat exchanger 1, fuel node 1                  = hx_fh1_f1
# e.g. helium->water heat eachanger 1, helium node 2, coolant loop = hx_hwc1_h1
# populate node names, then use names to generate wrapper functions for state variables

nodes = ["c_f1", "c_f2", "c_t1", "c_c1", "c_c2", "c_m",
         "hx_fh1_f1", "hx_fh1_f2", "hx_fh1_t1", "hx_fh1_h1", "hx_fh1_h2",
         "hx_fh2_f1", "hx_fh2_f2", "hx_fh2_t1", "hx_fh2_h1", "hx_fh2_h2",
         "hx_ch1_c1", "hx_ch1_c2", "hx_ch1_t1", "hx_ch1_h1", "hx_ch1_h2",
         "hx_ch2_c1", "hx_ch2_c2", "hx_ch2_t1", "hx_ch2_h1", "hx_ch2_h2",
         "hx_hwf1_h1", "hx_hwf1_h2", "hx_hwf1_t1", "hx_hwf1_w1", "hx_hwf1_w2",
         "hx_hwf2_h1", "hx_hwf2_h2", "hx_hwf2_t1", "hx_hwf2_w1", "hx_hwf2_w2",
         "hx_hwc1_h1", "hx_hwc1_h2", "hx_hwc1_t1", "hx_hwc1_w1", "hx_hwc1_w2",
         "hx_hwc2_h1", "hx_hwc2_h2", "hx_hwc2_t1", "hx_hwc2_w1", "hx_hwc2_w2",
         "n", "C1", "C2", "C3", "C4", "C5", "C6", "rho",]

# loop over names to create state variable functions dynammically 
for index, node in enumerate(nodes):
    # Using a nested function to correctly bind the current value of 'i'
    def create_lambda(i):
        return lambda tau=None: y(i, tau) if tau is not None else y(i)
    # Assigning the lambda function to a global variable
    globals()[node] = create_lambda(index)

Define system

In [3]:
# %%
# CORE
# fuel nodes
T_c_f1 = W_f/(m_f_c/2)*((hx_fh1_f2(t-tau_hx_c_f)+y(35,t-tau_hx_c_f))/2-c_f1()) + (k_f1*P*n()/mcp_f_c) + F*(hA_ft_c*(y(2)-c_f1())/mcp_f_c)         # T_cf1: y(0)
T_c_f2 = W_f/(m_f_c/2)*(c_f1()-c_f2()) + (k_f2*P*n()/mcp_f_c) + F*(hA_ft_c*(y(2)-c_f2())/mcp_f_c)                      # T_cf2: y(1)

# tubes
T_c_t1=F*((hA_ft_c/mcp_t_c)*((c_f1()-c_t1())+(c_f2()-c_t1()))+(hA_tc_c/mcp_t_c)*((c_c1()-c_t1())+(c_c2()-c_t1())))                 # T_c_t1: y(2)

# coolant 
T_c_c1 = W_c/(m_c_c/2)*((hx_ch1_c2(t-tau_hx_c_c)+hx_ch2_c2(t-tau_hx_c_c))/2-c_c1())+F*((hA_tc_c*(c_t1()-c_c1())/mcp_c_c)+((hA_mc_c)/mcp_c_c)*((c_m()-c_c1())))# T_c_c1: c_c1()
T_c_c2 = W_c/(m_c_c/2)*(c_c1()-c_c2()) + F*((hA_tc_c*(y(2)-c_c2())/mcp_c_c)+((hA_mc_c)/mcp_c_c)*((c_m()-c_c2())))            # T_c_c1: c_c2()  

# moderator 
T_c_m = F*((hA_mc_c/mcp_m_c)*((c_c1()-c_m())+(c_c2()-c_m())))+(k_m*P*n()/mcp_m_c)                                      #T_c_b: y(5)


# CONTINUE WITH FIND AND REPLACES 



In [4]:
# FUEL-HELIUM HX1
# fuel nodes
T_hfh_f1 = W_f/(m_f_hx)*(c_f2(t-tau_c_hx_f)-y(6)) + F*(hA_ft_hx*(y(8)-y(6))/mcp_f_hx)                               # T_cf1: y(6)
T_hfh_f2 = W_f/(m_f_hx)*(y(6)-hx_fh1_f2()) + F*(hA_ft_hx*(y(8)-hx_fh1_f2())/mcp_f_hx)                                            # T_cf2: y(7)

# tubes
T_hfh_t1=F*((hA_ft_hx/mcp_t_hx)*((y(6)-y(8))+(hx_fh1_f2()-y(8)))+(hA_ht_hx/mcp_t_hx)*((y(9)-y(8))+(y(10)-y(8))))            # T_ht1: y(8)

# helium
T_hfh_h1 = W_h_fh/(m_h_hxfh)*(y(45,t-tau_h)-y(9)) + F*(hA_ht_hx*(y(8)-y(9))/mcp_h_hxfh)                          # T_cc1: y(9) 
T_hfh_h2 = W_h_fh/(m_h_hxfh)*(y(9)-y(10)) + F*(hA_ht_hx*(y(8)-y(10))/mcp_h_hxfh)                                 # T_cc1: y(10) 

# FUEL-HELIUM HX2
# fuel nodes
T_hfh2_f1 = W_f/(m_f_hx)*(c_f2(t-tau_c_hx_f)-y(34)) + F*(hA_ft_hx*(y(36)-y(34))/mcp_f_hx)                           # T_cf1: y(34)
T_hfh2_f2 = W_f/(m_f_hx)*(y(34)-y(35)) + F*(hA_ft_hx*(y(36)-y(35))/mcp_f_hx)                                       # T_cf2: y(35)

# tubes
T_hfh2_t1=F*((hA_ft_hx/mcp_t_hx)*((y(34)-y(36))+(y(35)-y(36)))+(hA_ht_hx/mcp_t_hx)*((y(37)-y(36))+(y(38)-y(36))))    # T_ht1: y(36)

# helium
T_hfh2_h1 = W_h_fh/(m_h_hxfh)*(y(17)-y(37)) + F*(hA_ht_hx*(y(36)-y(37))/mcp_h_hxfh)                        # T_cc1: y(37) 
T_hfh2_h2 = W_h_fh/(m_h_hxfh)*(y(37)-y(38)) + F*(hA_ht_hx*(y(36)-y(38))/mcp_h_hxfh)                                 # T_cc1: y(38) 


In [5]:


# COOLANT-HELIUM HX1
# fuel nodes
T_hch_c1 = W_c/(m_c_hx)*(y(4,t-tau_c_hx_f)-y(11)) + F*(hA_ct_hx*(y(13)-y(11))/mcp_c_c)                        # T_cf1: y(11)
T_hch_c2 = W_c/(m_c_hx)*(y(11)-y(12)) + F*(hA_ct_hx*(y(13)-y(12))/mcp_c_c)                                    # T_cf2: y(12)

# tubes
T_hch_t1 = F*((hA_ct_hx/mcp_t_hx)*((y(11)-y(13))+(y(12)-y(13)))+(hA_th_hxch/mcp_t_hx)*((y(14)-y(13))+(y(15)-y(13))))   # T_ht1: y(13)

# helium
T_hch_h1 = W_h_ch/(m_h_hxfh/2)*(y(22,t-tau_h)-y(14)) + F*(hA_th_hxch*(y(13)-y(14))/mcp_h_hxfh)                   # T_cc1: y(14) 
T_hch_h2 = W_h_ch/(m_h_hxfh/2)*(y(14)-y(15)) + F*(hA_th_hxch*(y(13)-y(15))/mcp_h_hxfh)                           # T_cc1: y(15) 


# COOLANT-HELIUM HX2
# fuel nodes
T_hch2_c1 = W_c/(m_c_hx)*(y(4,t-tau_c_hx_f)-y(39)) + (hA_ct_hx*(y(41)-y(39))/mcp_c_c)                        # T_cf1: y(39)
T_hch2_c2 = W_c/(m_c_hx)*(y(39)-y(40)) + (hA_ct_hx*(y(41)-y(40))/mcp_c_c)                                    # T_cf2: y(40)

# tubes
T_hch2_t1 = F*((hA_ct_hx/mcp_t_hx)*((y(39)-y(41))+(y(40)-y(41)))+(hA_th_hxch/mcp_t_hx)*((y(42)-y(41))+(y(43)-y(41))))   # T_ht1: y(41)

# helium
T_hch2_h1 = W_h_ch/(m_h_hxfh)*(y(50,t)-y(42)) + F*(hA_th_hxch*(y(41)-y(42))/mcp_h_hxfh)                   # T_cc1: y(42) 
T_hch2_h2 = W_h_ch/(m_h_hxfh)*(y(42)-y(43)) + F*(hA_th_hxch*(y(41)-y(43))/mcp_h_hxfh)                           # T_cc1: y(43) 



In [6]:

# HELIUM-WATER HX1 (FUEL LOOP)
# helium
T_hhwf_h1 = W_h_fh/(m_h_hxhw/2)*(y(10)-y(16)) + F*(hA_ht_hxhw*(y(18)-y(16))/mcp_h_hxhw)                  # T_cf1: y(16)
T_hhwf_h2 = W_h_fh/(m_h_hxhw/2)*(y(16)-y(17)) + F*(hA_ht_hxhw*(y(18)-y(17))/mcp_h_hxhw)                          # T_cf2: y(17)

# tubes
T_hhwf_t1 = F*((hA_ht_hxhw/mcp_t_hxhw)*((y(16)-y(18))+(y(17)-y(18)))+(hA_tw_hxhw/mcp_t_hxhw)*((y(19)-y(18))+(y(20)-y(18)))) # T_ht1: y(18)

# water
T_hhwf_w1 = W_hhwf_w/(m_w/2)*(T0_hhwf_w1-y(19)) + F*(hA_tw_hxhw*(y(18)-y(19))/mcp_w)                                       # T_cc1: y(19) 
T_hhwf_w2 = W_hhwf_w/(m_w/2)*(y(19)-y(20)) + F*(hA_tw_hxhw*(y(18)-y(20))/mcp_w)                                            # T_cc1: y(20) 


# HELIUM-WATER HX2 (FUEL LOOP)
# helium
T_hhwf2_h1 = W_h_fh/(m_h_hxhw/2)*(y(38)-y(44)) + F*(hA_ht_hxhw*(y(46)-y(44))/mcp_h_hxhw)                       # T_cf1: y(44)
T_hhwf2_h2 = W_h_fh/(m_h_hxhw/2)*(y(44)-y(45)) + F*(hA_ht_hxhw*(y(46)-y(45))/mcp_h_hxhw)                                   # T_cf2: y(45)

# tubes
T_hhwf2_t1 = F*((hA_ht_hxhw/mcp_t_hxhw)*((y(44)-y(46))+(y(45)-y(46)))+(hA_tw_hxhw/mcp_t_hxhw)*((y(47)-y(46))+(y(48)-y(46)))) # T_ht1: y(46)

# water
T_hhwf2_w1 = W_hhwf_w/(m_w/2)*(T0_hhwf_w1-y(47)) + F*(hA_tw_hxhw*(y(46)-y(47))/mcp_w)                                       # T_cc1: y(47) 
T_hhwf2_w2 = W_hhwf_w/(m_w/2)*(y(47)-y(48)) + F*(hA_tw_hxhw*(y(46)-y(48))/mcp_w)                                            # T_cc1: y(48) 



In [7]:
# HELIUM-WATER HX (COOLANT LOOP)
# fuel nodes
T_hhwc_h1 = W_h_ch/(m_h_hxhw/2)*(y(15)-y(21)) + F*(hA_ht_hxhwc*(y(23)-y(21))/mcp_h_hxhw)       # T_cf1: y(21)
T_hhwc_h2 = W_h_ch/(m_h_hxhw/2)*(y(21)-y(22)) + F*(hA_ht_hxhwc*(y(23)-y(22))/mcp_h_hxhw)                                 # T_cf2: y(22)

# tubes
T_hhwc_t1 = F*((hA_ht_hxhwc/mcp_t_hxhw)*((y(21)-y(23))+(y(22)-y(23))) + (hA_tw_hxhw/mcp_t_hxhw)*((y(24)-y(23))+(y(25)-y(23)))) # T_ht1: y(23)

# water
T_hhwc_w1 = W_hhwc_w/(m_w/2)*(T0_hhwf_w1-y(24)) + F*(hA_tw_hxhw*(y(23)-y(24))/mcp_w)                                       # T_cc1: y(24) maybe don't need these nodes
T_hhwc_w2 = W_hhwc_w/(m_w/2)*(y(24)-y(25)) + F*(hA_tw_hxhw*(y(23)-y(25))/mcp_w)                                            # T_cc1: y(25) 

# HELIUM-WATER HX2 (COOLANT LOOP)
# fuel nodes
T_hhwc2_h1 = W_h_ch/(m_h_hxhw/2)*(y(43)-y(49)) + F*(hA_ht_hxhwc*(y(51)-y(49))/mcp_h_hxhw)       # T_cf1: y(49)
T_hhwc2_h2 = W_h_ch/(m_h_hxhw/2)*(y(49)-y(50)) + F*(hA_ht_hxhwc*(y(51)-y(50))/mcp_h_hxhw)                                 # T_cf2: y(50)

# tubes
T_hhwc2_t1 = F*(hA_ht_hxhwc/mcp_t_hxhw)*((y(49)-y(51))+(y(50)-y(51)))+(hA_tw_hxhw/mcp_t_hxhw)*((y(52)-y(51))+(y(53)-y(51))) # T_ht1: y(51)

# water
T_hhwc2_w1 = W_hhwc_w/(m_w/2)*(T0_hhwf_w1-y(52)) + F*(hA_tw_hxhw*(y(51)-y(52))/mcp_w)                                       # T_cc1: y(52) maybe don't need these nodes
T_hhwc2_w2 = W_hhwc_w/(m_w/2)*(y(52)-y(53)) + F*(hA_tw_hxhw*(y(51)-y(53))/mcp_w)                                            # T_cc1: y(53) 


In [8]:

dn = ((rho())-beta_t)*n()/Lam+lam[0]*C1()+lam[1]*C2()+lam[2]*C3()+lam[3]*C4()+lam[4]*C5()+lam[5]*C6()           # n (no source insertion): y(26)

# dC_i/dt (precursor concentrations)
dC1 = n()*beta[0]/Lam - lam[0]*C1() - C1()/tau_c + C1(t-tau_l)*np.exp(-lam[0]*tau_l)/tau_c                       # C1: y(27)
dC2 = n()*beta[1]/Lam - lam[1]*C2() - C2()/tau_c + C2(t-tau_l)*np.exp(-lam[1]*tau_l)/tau_c                       # C2: y(28)
dC3 = n()*beta[2]/Lam - lam[2]*C3() - C3()/tau_c + C3(t-tau_l)*np.exp(-lam[2]*tau_l)/tau_c                       # C3: y(29)
dC4 = n()*beta[3]/Lam - lam[3]*C4() - C4()/tau_c + C4(t-tau_l)*np.exp(-lam[3]*tau_l)/tau_c                       # C4: y(30)
dC5 = n()*beta[4]/Lam - lam[4]*C5() - C5()/tau_c + C5(t-tau_l)*np.exp(-lam[4]*tau_l)/tau_c                       # C5: y(31)
dC6 = n()*beta[5]/Lam - lam[5]*C6() - C6()/tau_c + C6(t-tau_l)*np.exp(-lam[5]*tau_l)/tau_c                       # C6: y(32)

# reactivity 
drho = (a_f/2)*(T_c_f1 + T_c_f2)+(a_b)*(T_c_m)+(a_c/2)*(T_c_c1+T_c_c2)           # rho: y(33)

Initial values & solve

In [9]:
# instantiate jitcdde object
DDE = jitcdde([T_c_f1,T_c_f2,T_c_t1,T_c_c1,T_c_c2,T_c_m,T_hfh_f1,T_hfh_f2,T_hfh_t1,T_hfh_h1,T_hfh_h2,
               T_hch_c1,T_hch_c2,T_hch_t1,T_hch_h1,T_hch_h2,T_hhwf_h1,T_hhwf_h2,T_hhwf_t1,T_hhwf_w1,T_hhwf_w2,
               T_hhwc_h1,T_hhwc_h2,T_hhwc_t1,T_hhwc_w1,T_hhwc_w2,dn,dC1,dC2,dC3,dC4,dC5,dC6,drho,T_hfh2_f1,T_hfh2_f2,T_hfh2_t1,
               T_hfh2_h1,T_hfh2_h2,T_hch2_c1,T_hch2_c2,T_hch2_t1,T_hch2_h1,T_hch2_h2,T_hhwf2_h1,T_hhwf2_h2,T_hhwf2_t1,
               T_hhwf2_w1,T_hhwf2_w2,T_hhwc2_h1,T_hhwc2_h2,T_hhwc2_t1,T_hhwc2_w1,T_hhwc2_w2])

# set initial conditions
DDE.constant_past([T0_c_f1,T0_c_f2,T0_c_t1,T0_c_c1,T0_c_c2,T0_c_m,T0_hfh_f1,T0_hfh_f2,T0_hfh_t1,T0_hfh_h1,T0_hfh_h2,
               T0_hch_c1,T0_hch_c2,T0_hch_t1,T0_hch_h1,T0_hch_h2,T0_hhwf_h1,T0_hhwf_h2,T0_hhwf_t1,T0_hhwf_w1,T0_hhwf_w2,
               T0_hhwc_h1,T0_hhwc_h2,T0_hhwc_t1,T0_hhwc_w1,T0_hhwc_w2,n_frac0,C0[0],C0[1],C0[2],C0[3],C0[4],C0[5],0.0,
               T0_hfh_f1,T0_hfh_f2,T0_hfh_t1,T0_hfh_h1,T0_hfh_h2,T0_hch_c1,T0_hch_c2,T0_hch_t1,T0_hch_h1,T0_hch_h2,
               T0_hhwf_h1+200,T0_hhwf_h2+200,T0_hhwf_t1,T0_hhwf_w1,T0_hhwf_w2,T0_hhwc_h1,T0_hhwc_h2,T0_hhwc_t1,T0_hhwc_w1,T0_hhwc_w2])

#DDE.set_integration_parameters(atol=1e-10, rtol=1e-05, first_step=1.0, min_step=1e-11, max_step=10.0, decrease_threshold=1.1, 
#                           increase_threshold=0.5, safety_factor=0.9, max_factor=5.0, min_factor=0.2, pws_factor=3, 
#                           pws_atol=0.0, pws_rtol=1e-05, pws_max_iterations=10, pws_base_increase_chance=0.1, pws_fuzzy_increase=False)
#
# DDE.step_on_discontinuities()

# jitcdde solver parameters 
t0 = 0.0
tf = 1000.00
T = np.arange(t0,tf,0.01)

sol_jit = []
for t_x in T:
    sol_jit.append(DDE.integrate(t_x))

Generating, compiling, and loading C code.
Using default integration parameters.


/mnt/envs/thesis_env/lib/python3.9/site-packages/jitcdde/_jitcdde.py:795: UserWarning: You did not explicitly handle initial discontinuities. Proceed only if you know what you are doing. This is only fine if you somehow chose your initial past such that the derivative of the last anchor complies with the DDE. In this case, you can set the attribute `initial_discontinuities_handled` to `True` to suppress this warning. See https://jitcdde.rtfd.io/#discontinuities for details.
  warn("You did not explicitly handle initial discontinuities. Proceed only if you know what you are doing. This is only fine if you somehow chose your initial past such that the derivative of the last anchor complies with the DDE. In this case, you can set the attribute `initial_discontinuities_handled` to `True` to suppress this warning. See https://jitcdde.rtfd.io/#discontinuities for details.")
/mnt/envs/thesis_env/lib/python3.9/site-packages/jitcdde/_jitcdde.py:795: UserWarning: You did not explicitly handle in

UnsuccessfulIntegration: 
Could not integrate with the given tolerance parameters:

atol: 1.000000e-10
rtol: 1.000000e-05
min_step: 1.000000e-10

The most likely reasons for this are:
• You did not sufficiently address initial discontinuities. See https://jitcdde.rtfd.io/#discontinuities for details.
• The DDE is ill-posed or stiff.

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(18,18))

t_0 = 0.0
t_f = 250

p_error = 0.17

axs[0,0].set_xlim([t_0,t_f])
axs[0,0].plot(T,[s[0] for s in sol_jit],label="core f1") 
axs[0,0].plot(T,[s[1] for s in sol_jit],label="core f2") 
axs[0,0].plot(T,[s[2] for s in sol_jit],label="core t1")  
axs[0,0].plot(T,[s[3] for s in sol_jit],label="core c1")
axs[0,0].plot(T,[s[4] for s in sol_jit],label="core c2")
axs[0,0].plot(T,[s[5] for s in sol_jit],label="core m")
axs[0,0].legend()
axs[0,0].set_title("Core Node Temperatures (K)")
axs[0,0].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off


axs[0,1].set_xlim([t_0,t_f])
# axs[0,1].set_ylim([650,750])
axs[0,1].plot(T,[s[6] for s in sol_jit],label="hx_fh f1") 
axs[0,1].plot(T,[s[7] for s in sol_jit],label="hx_fh f2") 
axs[0,1].plot(T,[s[8] for s in sol_jit],label="hx_fh t1")  
axs[0,1].plot(T,[s[9] for s in sol_jit],label="hx_fh h1")
axs[0,1].plot(T,[s[10] for s in sol_jit],label="hx_fh h2")
axs[0,1].plot(T,[s[34] for s in sol_jit],label="hx_fh2 f1") 
axs[0,1].plot(T,[s[35] for s in sol_jit],label="hx_fh2 f2") 
axs[0,1].plot(T,[s[36] for s in sol_jit],label="hx_fh2 t1")  
axs[0,1].plot(T,[s[37] for s in sol_jit],label="hx_fh2 h1")
axs[0,1].plot(T,[s[38] for s in sol_jit],label="hx_fh2 h2")

axs[0,1].legend()
axs[0,1].set_title("HX Fuel->Helium Node Temperatures (K)")
#axs[0,1].tick_params(
#    axis='x',          # changes apply to the x-axis
#    which='both',      # both major and minor ticks are affected
#    bottom=False,      # ticks along the bottom edge are off
#    top=False,         # ticks along the top edge are off
#    labelbottom=False) # labels along the bottom edge are off

axs[0,2].set_xlim([t_0,t_f])
axs[0,2].plot(T,[s[16] for s in sol_jit],label="hx_hwf h1") 
axs[0,2].plot(T,[s[17] for s in sol_jit],label="hx_hwf h2") 
axs[0,2].plot(T,[s[18] for s in sol_jit],label="hx_hwf t1")  
axs[0,2].plot(T,[s[19] for s in sol_jit],label="hx_hwf w1")
axs[0,2].plot(T,[s[20] for s in sol_jit],label="hx_hwf w2")
axs[0,2].plot(T,[s[44] for s in sol_jit],label="hx_hwf2 h1") 
axs[0,2].plot(T,[s[45] for s in sol_jit],label="hx_hwf2 h2") 
axs[0,2].plot(T,[s[46] for s in sol_jit],label="hx_hwf2 t1")  
axs[0,2].plot(T,[s[47] for s in sol_jit],label="hx_hwf2 w1")
axs[0,2].plot(T,[s[48] for s in sol_jit],label="hx_hwf2 w2")
axs[0,2].legend()
axs[0,2].set_title("HX Helium->Water (Fuel Loop) Node Temperatures (K)")
axs[0,2].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off

axs[1,1].set_xlim([t_0,t_f])
axs[1,1].plot(T,[s[11] for s in sol_jit],label="hx_ch f1") 
axs[1,1].plot(T,[s[12] for s in sol_jit],label="hx_ch f2") 
axs[1,1].plot(T,[s[13] for s in sol_jit],label="hx_ch t1")  
axs[1,1].plot(T,[s[14] for s in sol_jit],label="hx_ch h1")
axs[1,1].plot(T,[s[15] for s in sol_jit],label="hx_ch h2")
axs[1,1].plot(T,[s[39] for s in sol_jit],label="hx_ch f1") 
axs[1,1].plot(T,[s[40] for s in sol_jit],label="hx_ch f2") 
axs[1,1].plot(T,[s[41] for s in sol_jit],label="hx_ch t1")  
axs[1,1].plot(T,[s[42] for s in sol_jit],label="hx_ch h1")
axs[1,1].plot(T,[s[43] for s in sol_jit],label="hx_ch h2")
axs[1,1].legend()
axs[1,1].legend()
axs[1,1].set_title("HX Coolant->Helium Node Temperatures (K)")

# fuel temps
axs[1,2].set_xlim([t_0,t_f])
axs[1,2].plot(T,[s[21] for s in sol_jit],label="hx_hwc h1") 
axs[1,2].plot(T,[s[22] for s in sol_jit],label="hx_hwc h2") 
axs[1,2].plot(T,[s[23] for s in sol_jit],label="hx_hwc t1")  
axs[1,2].plot(T,[s[24] for s in sol_jit],label="hx_hwc w1")
axs[1,2].plot(T,[s[25] for s in sol_jit],label="hx_hwc w2")
axs[1,2].plot(T,[s[49] for s in sol_jit],label="hx_hwc h1") 
axs[1,2].plot(T,[s[50] for s in sol_jit],label="hx_hwc h2") 
axs[1,2].plot(T,[s[51] for s in sol_jit],label="hx_hwc t1")  
axs[1,2].plot(T,[s[52] for s in sol_jit],label="hx_hwc w1")
axs[1,2].plot(T,[s[53] for s in sol_jit],label="hx_hwc w2")
axs[1,2].legend()
axs[1,2].set_title("HX Helium->Water (Coolant Loop) Node Temperatures (K)")

axs[1,0].plot(T,[s[26] for s in sol_jit],label="n") 
axs[1,0].plot(T,[(1+p_error)*s[26] for s in sol_jit],linestyle="--",label="+20%") 
axs[1,0].plot(T,[(1-p_error)*s[26] for s in sol_jit],linestyle="--",label="-20%") 
axs[1,0].set_xlabel("t (s)")
axs[1,0].set_title(r"$n$")
axs[1,0].set_ylabel(r"$\frac{n}{n_0}$")
axs[1,0].set_xlim([t_0,t_f])
axs[1,0].legend()

axs[2,0].plot(T,[s[33] for s in sol_jit],label="n") 
axs[2,0].set_xlabel("t (s)")
axs[2,0].set_title(r"$\rho$")
axs[2,0].set_xlim([t_0,t_f])

axs[2,1].plot(T,[s[27] for s in sol_jit],label="C1") 
axs[2,1].plot(T,[s[28] for s in sol_jit],label="C2") 
axs[2,1].plot(T,[s[29] for s in sol_jit],label="C3")  
axs[2,1].plot(T,[s[30] for s in sol_jit],label="C4")
axs[2,1].plot(T,[s[31] for s in sol_jit],label="C5")
axs[2,1].plot(T,[s[32] for s in sol_jit],label="C6")
axs[2,1].legend()
axs[2,1].set_xlabel("t (s)")
axs[2,1].set_yscale("log")
axs[2,1].set_ylabel(r"concentration (1/cm$^3$)")
axs[2,1].legend(loc="right")
axs[2,1].set_title("Precursor Concentrations")
axs[2,1].set_xlim([t_0,t_f])


In [ ]:
print((1-p_error)*sol_jit[-1][26])
print(sol_jit[-1][26])
print((1+p_error)*sol_jit[-1][26])
print(sol_jit[33][-1])

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(16,6))

axs[0].set_xlim([t_0,10])
# axs[0,1].set_ylim([650,750])
axs[0].plot(T,[s[6] for s in sol_jit],label="hx_fh f1") 
axs[0].plot(T,[s[7] for s in sol_jit],label="hx_fh f2") 
axs[0].plot(T,[s[8] for s in sol_jit],label="hx_fh t1")  
axs[0].plot(T,[s[9] for s in sol_jit],label="hx_fh h1")
axs[0].plot(T,[s[10] for s in sol_jit],label="hx_fh h2")
axs[0].plot(T,[s[34] for s in sol_jit],label="hx_fh2 f1") 
axs[0].plot(T,[s[35] for s in sol_jit],label="hx_fh2 f2") 
axs[0].plot(T,[s[36] for s in sol_jit],label="hx_fh2 t1")  
axs[0].plot(T,[s[37] for s in sol_jit],label="hx_fh2 h1")
axs[0].plot(T,[s[38] for s in sol_jit],label="hx_fh2 h2")
axs[0].set_title("HX Fuel->Helium Node Temperatures (K)")
axs[0].legend(loc="right")
axs[0].set_xlabel(r"$t$(s)")

axs[1].set_xlim([t_0,50])
axs[1].plot(T,[s[16] for s in sol_jit],label="hx_hwf h1") 
axs[1].plot(T,[s[17] for s in sol_jit],label="hx_hwf h2") 
axs[1].plot(T,[s[18] for s in sol_jit],label="hx_hwf t1")  
axs[1].plot(T,[s[19] for s in sol_jit],label="hx_hwf w1")
axs[1].plot(T,[s[20] for s in sol_jit],label="hx_hwf w2")
axs[1].plot(T,[s[44] for s in sol_jit],label="hx_hwf2 h1") 
axs[1].plot(T,[s[45] for s in sol_jit],label="hx_hwf2 h2") 
axs[1].plot(T,[s[46] for s in sol_jit],label="hx_hwf2 t1")  
axs[1].plot(T,[s[47] for s in sol_jit],label="hx_hwf2 w1")
axs[1].plot(T,[s[48] for s in sol_jit],label="hx_hwf2 w2")
axs[1].set_title("HX Helium->Water (Fuel Loop) Node Temperatures (K)")
axs[1].legend()
axs[1].set_xlabel(r"$t$(s)")

In [ ]:
# power generation from fission 

# core power generation
pow_f1 = [k_f1*P*s[26] for s in sol_jit]
pow_f2 = [k_f2*P*s[26] for s in sol_jit]
tot_pow = np.add(pow_f1,pow_f2)

# heat extracted by fuel-helium hx
fh_ex = [(s[1]-s[0])*W_f*scp_f for s in sol_jit]

# heat extracted by coolant-helium hx
ch_ex = [(s[4]-s[3])*W_c*scp_c for s in sol_jit]

# helium-> water (fuel loop)
hwf_ex = [(s[10]-s[9])*W_h_fh*scp_h for s in sol_jit]

# helium-> water (coolant loop)
hwc_ex = [(s[15]-s[14])*W_h_ch*scp_h for s in sol_jit]

tot_ex = [fh_ex[i]+ch_ex[i]+hwf_ex[i]+hwc_ex[i] for i in range(len(sol_jit))]

plt.plot(T,tot_pow,label="core power generation")
plt.plot(T,fh_ex,label="fh hx extraction")
plt.plot(T,ch_ex,label="ch hx extraction")
plt.plot(T,hwf_ex,label="hwf hx extraction")
plt.plot(T,hwc_ex,label="hwc hx extraction")
plt.plot(T,tot_ex,label="total extraction",linestyle="--")
plt.legend()
